# Collect Semantic Model information
This notebook is intended to collect information from all semantic models that are located in the referenced workspace. 
All information collected will be saved to the connected lakehouse (default lakehouse of this notebook) and stored for analysis and reporting. 

##### What's included
The notebook loops over all semantic models in the workspace and collects information about: 
- Tables
- Columns
- Relationships between tables
- Measures

##### Not included
Any other additional model objects or pieces of information (like partitions, storage modes, field parameters, refresh policies etc) are currently out of scope and can be added in the future. 

##### Missing functionality / room for improvement
The current version of the notebook does not allow setups like slowly changing dimensions. Currently, all tables are truncated before collecting information. Therefore, it is not (yet) possible to travel in time and see how a definition of a measure changed of time (for example). 


#### Define parameters

In [1]:
workspace_name = "Semantic Link for Power BI Folks" # not used currently, scoped to current workspace.
lakehouse_name = "ModelDocumenter"

StatementMeta(, 9aad1dcf-1206-499e-b3fa-8ffaa9520779, 3, Finished, Available, Finished)

#### Import libraries

In [2]:
# Set the bases
from datetime import datetime, timezone
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window
run_timestamp = datetime.now(tz=timezone.utc).isoformat()
import pyspark.sql.functions as F
import sempy.fabric as fabric
import time

StatementMeta(, 9aad1dcf-1206-499e-b3fa-8ffaa9520779, 4, Finished, Available, Finished)

#### Clean-up lakehouse
This section cleans-up the lakehouse before new information is collected. 

<mark>**Note:** in the current version, there is no collection of historical information in the lakehouse due to the trucate that happens before collecting new information.  </mark>

In [3]:
# To avoid duplications and polution in lakehouse, truncate all tables

# List all tables in the default database
tables_df = spark.sql("SHOW TABLES")

# Collect the table names
tables = tables_df.collect()

# Iterate through each table and execute TRUNCATE statements
for table in tables:
    table_name = table.tableName
    print(f"Truncating table: {table_name}")
    
    # Execute TRUNCATE command
    spark.sql(f" DELETE FROM {table_name}")

StatementMeta(, 9aad1dcf-1206-499e-b3fa-8ffaa9520779, 5, Finished, Available, Finished)

Truncating table: md_datasets
Truncating table: md_tables
Truncating table: md_columns
Truncating table: md_relationships
Truncating table: md_measures


#### Functions
As the information retrieved from the semantic models can contain spaces in table/column/object names, which are not allowed in lakehouses, these functions help to remove special characters from object names. 

The second function is created to save the retrieved information to the lakehouse and adding additional columns (such as semantic model name and id) to support scenarios in which relationships between the tables are desired (in case multiple semantic models are in scope)

In [4]:
# Function to get rid of special characters in retrieved information
def remove_special_chars_from_col_names(dataframe, tokens = r" .,;{}()\="):
    cols_without_bad_characters = [
        col.translate({ord(token): None for token in tokens})
        for col in dataframe.columns
    ]
    return dataframe.toDF(*cols_without_bad_characters)

# Function to save the result set to the destination lakehouse, and adding additional information to the resultset such as DatasetName, DatasetId and a timestamp
def save_table(dataframe, dataset_name, entity):
    if dataframe.empty: 
        print(f"no results found for {entity}")
        return 
    df = spark.createDataFrame(dataframe)
    df = remove_special_chars_from_col_names(df)
    df = df.withColumn("DatasetName", F.lit(dataset_name))
    df = df.withColumn("DatasetId", F.lit(dataset_id))
    df = df.withColumn("_load_datetime", F.lit(run_timestamp))
    print(f"Writing entity '{entity}' to table {lakehouse_name}.MD_{entity}")
    df.write.mode("append").saveAsTable(f"{lakehouse_name}.MD_{entity}")

StatementMeta(, 9aad1dcf-1206-499e-b3fa-8ffaa9520779, 6, Finished, Available, Finished)

## Main code
This section references above defined parameters and functions and collects the actual information. 

#### Collect semantic models in workspace

In [5]:
# List all semantic models in workspace
semanticmodels = fabric.list_datasets()
semanticmodels[['Dataset Name', 'Dataset ID']] # in case filter add: .str.endswith('_SM')]

# Create datasets table
df_datasets = spark.createDataFrame(semanticmodels)
#df_datasets.write.format("delta").mode("overwrite").save(lakehouse_name)

StatementMeta(, 9aad1dcf-1206-499e-b3fa-8ffaa9520779, 7, Finished, Available, Finished)

In [6]:
# Save datasets table to lakehouse
df_datasets = remove_special_chars_from_col_names(df_datasets)
df_datasets.write.mode("append").saveAsTable("MD_datasets")

StatementMeta(, 9aad1dcf-1206-499e-b3fa-8ffaa9520779, 8, Finished, Available, Finished)

#### Avoid duplicate rows in lakehouse
Scans the lakehouse table and remove duplicates from the semantic models / datasets table

In [7]:
# Clean-up dataset list

# Get current list of datasets
cleandatsets = spark.sql(f"SELECT * FROM {lakehouse_name}.md_datasets")
cleandatasetsrowcount = cleandatsets.count()

#from pyspark.sql.functions import col, row_number
#from pyspark.sql.window import Window

# Define the window specification to partition by Dataset ID and order by Last Update in descending order
window_spec = Window.partitionBy("DatasetID").orderBy(col("LastUpdate").desc())

# Add a row number column to identify the most recent record
df_with_row_number = cleandatsets.withColumn("row_number", row_number().over(window_spec))

# Filter to keep only the most recent record for each Dataset ID
df_no_duplicates = df_with_row_number.filter(col("row_number") == 1).drop("row_number")
noduplicatesrowcount = df_no_duplicates.count()

# Show rowcount diff
print(f"Original row count: {cleandatasetsrowcount}, cleaned set: {noduplicatesrowcount}")

# save updated datasets list
df_no_duplicates.write.mode("overwrite").saveAsTable("MD_datasets")
updatedrowcount = spark.sql(f"SELECT COUNT(*) AS VALIDATEDROWCOUNTAFTERUPDATE FROM {lakehouse_name}.md_datasets")
print("Table cleaned successfully")

StatementMeta(, 9aad1dcf-1206-499e-b3fa-8ffaa9520779, 9, Finished, Available, Finished)

Original row count: 8, cleaned set: 8
Table cleaned successfully


#### Collect additional information
This section loops over each semantic model in the workspace (as collected before) and collects additional information, such as: 
- Table information
- Column information
- Relationship information
- Measure information

Each of these resultsets is saved in a separate table in the lakehouse (using the earlier defined functions). 

##### Missing information
Some semantic models (such as default semantic models) may not contain all pieces of information or a semantic model might not have measures for example. The logging of this step will always return what it found in the semantic model and will inform in case there are no results. 

In [8]:
# For each semantic model
for index, row in semanticmodels.iterrows():

    # Extract dataset_name and dataset_id from the current row
    dataset_name = row['Dataset Name']
    dataset_id = row['Dataset ID']

    # Print name and datset id to screen at start
    print(f"Starting Dataset Name: {dataset_name}, Dataset ID: {row['Dataset ID']}...")

    # Get semantic model meta data and save to lakehouse
    save_table(fabric.list_tables(dataset_name), dataset_name, "tables")
    save_table(fabric.list_tables(dataset_name, include_columns=True), dataset_name, "columns")
    save_table(fabric.list_relationships(dataset_name), dataset_name, "relationships")
    save_table(fabric.list_measures(dataset_name), dataset_name, "measures")

    # Print name and dataset id to screen after completion
    print(f"...Finished Dataset Name: {dataset_name}, Dataset ID: {dataset_id}")

StatementMeta(, 9aad1dcf-1206-499e-b3fa-8ffaa9520779, 10, Finished, Available, Finished)

Starting Dataset Name: SempyDemo, Dataset ID: 20512a78-371d-4a9f-a6cc-c806ee080040...
no results found for tables
no results found for columns
no results found for relationships
no results found for measures
...Finished Dataset Name: SempyDemo, Dataset ID: 20512a78-371d-4a9f-a6cc-c806ee080040
Starting Dataset Name: Example Semantic Model, Dataset ID: 8ef42108-5df7-4222-85ee-76d8b8c40bc5...
Writing entity 'tables' to table ModelDocumenter.MD_tables
Writing entity 'columns' to table ModelDocumenter.MD_columns
Writing entity 'relationships' to table ModelDocumenter.MD_relationships
Writing entity 'measures' to table ModelDocumenter.MD_measures
...Finished Dataset Name: Example Semantic Model, Dataset ID: 8ef42108-5df7-4222-85ee-76d8b8c40bc5
Starting Dataset Name: ModelDocumentation, Dataset ID: e26c306a-7c56-4ddc-a1e5-28ca7b2bfa47...
Writing entity 'tables' to table ModelDocumenter.MD_tables
Writing entity 'columns' to table ModelDocumenter.MD_columns
Writing entity 'relationships' to tab